In [1]:
import pandas as pd
import os,glob
# 读取.dta文件
df = pd.read_stata("D:\文档\坚果云\区域经济\数字经济\数字经济指数\数据\专利\关键数字技术专利IPC对照表.dta")
df

,技术分支编号,技术分支名称,国际专利分类,关键词概述,关键数字技术ID
0,（一）人工智能技术专利分类体系表,,,,1.0
1,1,人工智能硬件平台,,,2.0
2,1.1,智能芯片,G06F3*、G06F8*、G06F9*、G06F11*、G06F12*、G06F13*、G...,智能集成电路、智能芯片、AI芯片、智能单片机、GPU、FPGA、ASIC、SOC芯片、类脑芯...,3.0
3,1.1.1,GPU,G06F9*、G06N3*、G06T1*、G06T3*、G06T5*、G06T7*、G06T...,GPU、图形处理器、图像处理器、视觉处理器、显示卡芯片、显卡芯片等,4.0
4,1.1.2,FPGA,G05B19*、G06F7*、G06F9*、G06F11*、G06F40*、G06F13*、...,FPGA、现场可编程逻辑器件、现场可编程门阵列、现场可编程逻辑门阵列等,5.0
...,...,...,...,...,...
587,3.5.6.2,联邦学习,G06N3/08、G06N20*、G06N99*,联邦学习、联邦建模、横向联邦、纵向联邦、联邦迁移学习,588.0
588,3.5.6.3,机密计算,G06F21/57、G06F21/60、G06F21/62、H04L9*,机密计算、可信执行环境、系统隔离,589.0
589,3.5.6.4,差分隐私,G06F21/62、H04L9*,差分隐私、本地化差分、中心化差分、分布式差分、隐私损失,590.0
590,3.5.6.5,同态加密,G06F21/62、H04L9*,加法同态、乘法加密、部分同态、全同态、PHE、SWHE、FHE,591.0


In [2]:
import re

# 提取所有IPC分类号范围（忽略空值）
ipc_patterns = []
for ipc_str in df['国际专利分类'].dropna():
    # 分割多个IPC（如 "G06F3*、G06F8*"）
    patterns = re.split(r'[、，, ]+', ipc_str.strip())
    ipc_patterns.extend(patterns)
ipc_patterns = [p.strip() for p in ipc_patterns if p.strip()]
# 转换为正则表达式（处理通配符*）
ipc_regex = []
for pattern in ipc_patterns:
    # 将通配符*替换为正则表达式.*，并确保匹配完整分类号
    regex_pattern = re.escape(pattern).replace(r'\*', '.*') + r'(\b|$)'
    ipc_regex.append(regex_pattern)

# 合并所有正则表达式为一个
combined_regex = '|'.join(ipc_regex)
combined_regex

'G06F3.*(\\b|$)|G06F8.*(\\b|$)|G06F9.*(\\b|$)|G06F11.*(\\b|$)|G06F12.*(\\b|$)|G06F13.*(\\b|$)|G06F15.*(\\b|$)|G06F16.*(\\b|$)|G06F17.*(\\b|$)|G06F21.*(\\b|$)|G06F30.*(\\b|$)|G06F40.*(\\b|$)|G06K7.*(\\b|$)|G06K9.*(\\b|$)|G06K17.*(\\b|$)|G06K19.*(\\b|$)|G06N.*(\\b|$)|G06T1.*(\\b|$)|G06T3.*(\\b|$)|G06T5.*(\\b|$)|G06T7.*(\\b|$)|G06T11.*(\\b|$)|G06T15.*(\\b|$)|G06V.*(\\b|$)|G16B.*(\\b|$)|G16C.*(\\b|$)|G16H.*(\\b|$)|H01L21.*(\\b|$)|H01L23.*(\\b|$)|H01L25.*(\\b|$)|H01L27.*(\\b|$)|H05K1.*(\\b|$)|H05K3.*(\\b|$)|G06F9.*(\\b|$)|G06N3.*(\\b|$)|G06T1.*(\\b|$)|G06T3.*(\\b|$)|G06T5.*(\\b|$)|G06T7.*(\\b|$)|G06T11.*(\\b|$)|G06T15.*(\\b|$)|G05B19.*(\\b|$)|G06F7.*(\\b|$)|G06F9.*(\\b|$)|G06F11.*(\\b|$)|G06F40.*(\\b|$)|G06F13.*(\\b|$)|G06F15.*(\\b|$)|G06F17.*(\\b|$)|G06F30.*(\\b|$)|H03K19.*(\\b|$)|G06F.*(\\b|$)|H01L21.*(\\b|$)|H01L23.*(\\b|$)|H01L25.*(\\b|$)|H01L27.*(\\b|$)|H03K.*(\\b|$)|H05K1.*(\\b|$)|H05K3.*(\\b|$)|G06N3.*(\\b|$)|G06F9.*(\\b|$)|G06F15.*(\\b|$)|G11C13.*(\\b|$)|G06N3.*(\\b|$)|G06F9.*(\\b|$

In [3]:
def is_digital_patent(ipc_str):
    if pd.isna(ipc_str):
        return False
    # 分割专利的多个IPC（如 "G06N3/08;G06N3/0464"）
    ipcs = re.split(r'[;，, ]+', ipc_str.strip())
    # 检查是否至少有一个IPC匹配数字技术范围
    for ipc in ipcs:
        if re.search(combined_regex, ipc, flags=re.IGNORECASE):
            return True
    return False

In [9]:
df_z_path = 'D:\文档\坚果云\区域经济\数字经济\数字经济指数\数据\专利\专利系统'
save_path = 'D:\文档\坚果云\区域经济\数字经济\数字经济指数\数据\专利\福建数字专利2024.csv'
csv_files = glob.glob(os.path.join(df_z_path, "**/*.xlsx"), recursive=True)
i = True
for csv_file in csv_files:
    print(csv_file)
    df_z = pd.read_excel(csv_file)
    df_z.dropna(subset=['IPC分类号'], inplace=True)
    df_z = df_z[['申请号', '公开（公告）号', '申请日', '公开（公告）日', 'IPC分类号', '申请（专利权）人','发明名称','申请人邮编','文献类型']]
    # 应用筛选
    df_digital_patents = df_z[df_z['IPC分类号'].apply(is_digital_patent)].copy()
    if i:
        df_digital_patents.to_csv(save_path,index=False)
    else:
        df_digital_patents.to_csv(save_path, index=False, header=None,mode='a')
    i = False

D:\文档\坚果云\区域经济\数字经济\数字经济指数\数据\专利\专利系统\202401-202402.xlsx


d:\Software\anaconda3\envs\gis\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


D:\文档\坚果云\区域经济\数字经济\数字经济指数\数据\专利\专利系统\202403-20240415.xlsx


d:\Software\anaconda3\envs\gis\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


D:\文档\坚果云\区域经济\数字经济\数字经济指数\数据\专利\专利系统\20240416-202405.xlsx


...

d:\Software\anaconda3\envs\gis\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


D:\文档\坚果云\区域经济\数字经济\数字经济指数\数据\专利\专利系统\20240730-202410.xlsx


d:\Software\anaconda3\envs\gis\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


D:\文档\坚果云\区域经济\数字经济\数字经济指数\数据\专利\专利系统\202411-202412.xlsx


d:\Software\anaconda3\envs\gis\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [15]:
df_z = pd.read_csv("D:\文档\坚果云\区域经济\数字经济\数字经济指数\数据\专利\福建数字专利2024.csv")
df_z

,申请号,公开（公告）号,申请日,公开（公告）日,IPC分类号,申请（专利权）人,发明名称,申请人邮编,文献类型
0,CN202410012063.2,CN118135171A,2024.01.03,2024.06.04,G06N3/08;G06N3/0464;G06V20/70;G06V10/10;G06V10/82,新大陆数字技术股份有限公司,一种基于深度学习的磁翻板液位仪自动预警方法,350015.0,发明
1,CN202410015335.4,CN118038893A,2024.01.05,2024.05.14,G10L25/03;G10L15/06;G10L25/27;G10L25/51,厦门快商通科技股份有限公司,基于声纹聚类的阈值选取方法、声纹聚类方法及装置,361000.0,发明
2,CN202410048496.3,CN117833943A,2024.01.12,2024.04.05,H04B1/16,数字蓝盾（厦门）信息科技有限公司,一种基于软件定义无线电的宽带信号接收装置及接收处理方法,361101.0,发明
3,CN202410032903.1,CN117855767A,2024.01.09,2024.04.09,H01M50/586;H01M50/178;B32B27/08;B32B7/12;H01M5...,宁德新能源科技有限公司,绝缘件、极耳组件、电芯、电池及用电设备,352100.0,发明
4,CN202410038215.6,CN117896881A,2024.01.10,2024.04.16,H05B45/30;H05B45/50;H05B47/155;H02J7/00,泉州海川半导体有限公司,一种用于移动电源外部LED灯数量的自动识别方法,362100.0,发明
...,...,...,...,...,...,...,...,...,...
27333,CN202411989720.5,CN119889437A,2024.12.31,2025.04.25,G16B20/00;G16B20/20;G16B20/50,厦门艾德生物医药科技股份有限公司,用于NGS样本污染的检测方法及其系统,361027.0,发明
27334,CN202411996494.3,CN119812685A,2024.12.31,2025.04.11,H05K1/02;H05K1/14;H01M50/503;H01M50/519;H01R12/71,厦门新能达科技有限公司,电池组及用电装置,361000.0,发明
27335,CN202411997100.6,CN119541383A,2024.12.31,2025.02.28,H10H29/49;G09G3/32;H10D86/60;H10H29/30;H10K59/...,天马新型显示技术研究院（厦门）有限公司; 厦门天马微电子有限公司,一种阵列基板、显示面板及拼接显示屏,361101.0,发明
27336,CN202411997340.6,CN119694253A,2024.12.31,2025.03.25,G09G3/3225,厦门天马显示科技有限公司,显示面板及显示装置,361101.0,发明


In [16]:
df_z['申请人邮编'] = df_z['申请人邮编'].astype(int)
df_z

,申请号,公开（公告）号,申请日,公开（公告）日,IPC分类号,申请（专利权）人,发明名称,申请人邮编,文献类型
0,CN202410012063.2,CN118135171A,2024.01.03,2024.06.04,G06N3/08;G06N3/0464;G06V20/70;G06V10/10;G06V10/82,新大陆数字技术股份有限公司,一种基于深度学习的磁翻板液位仪自动预警方法,350015,发明
1,CN202410015335.4,CN118038893A,2024.01.05,2024.05.14,G10L25/03;G10L15/06;G10L25/27;G10L25/51,厦门快商通科技股份有限公司,基于声纹聚类的阈值选取方法、声纹聚类方法及装置,361000,发明
2,CN202410048496.3,CN117833943A,2024.01.12,2024.04.05,H04B1/16,数字蓝盾（厦门）信息科技有限公司,一种基于软件定义无线电的宽带信号接收装置及接收处理方法,361101,发明
3,CN202410032903.1,CN117855767A,2024.01.09,2024.04.09,H01M50/586;H01M50/178;B32B27/08;B32B7/12;H01M5...,宁德新能源科技有限公司,绝缘件、极耳组件、电芯、电池及用电设备,352100,发明
4,CN202410038215.6,CN117896881A,2024.01.10,2024.04.16,H05B45/30;H05B45/50;H05B47/155;H02J7/00,泉州海川半导体有限公司,一种用于移动电源外部LED灯数量的自动识别方法,362100,发明
...,...,...,...,...,...,...,...,...,...
27333,CN202411989720.5,CN119889437A,2024.12.31,2025.04.25,G16B20/00;G16B20/20;G16B20/50,厦门艾德生物医药科技股份有限公司,用于NGS样本污染的检测方法及其系统,361027,发明
27334,CN202411996494.3,CN119812685A,2024.12.31,2025.04.11,H05K1/02;H05K1/14;H01M50/503;H01M50/519;H01R12/71,厦门新能达科技有限公司,电池组及用电装置,361000,发明
27335,CN202411997100.6,CN119541383A,2024.12.31,2025.02.28,H10H29/49;G09G3/32;H10D86/60;H10H29/30;H10K59/...,天马新型显示技术研究院（厦门）有限公司; 厦门天马微电子有限公司,一种阵列基板、显示面板及拼接显示屏,361101,发明
27336,CN202411997340.6,CN119694253A,2024.12.31,2025.03.25,G09G3/3225,厦门天马显示科技有限公司,显示面板及显示装置,361101,发明


In [24]:
import pandas as pd

# 1. 构建福建省邮编-市级映射表（县级自动归属到地级市）
def create_fujian_city_map():
    """生成邮编到地级市的映射表"""
    data = {
# 福州市（35开头）
    '3500': '福州市', '3501': '福州市',  # 鼓楼/台江/仓山/马尾
    '3502': '福州市',  # 晋安区
    '3503': '福州市',  # 福清市
    '3504': '福州市',  # 闽侯县
    '3505': '福州市',  # 连江县
    '3506': '福州市',  # 罗源县
    '3507': '福州市',  # 闽清县
    '3508': '福州市',  # 永泰县
    '3509': '福州市',  # 平潭县（平潭综合实验区）
    
    # 厦门市（36开头）
    '3610': '厦门市',  # 思明/湖里区
    '3611': '厦门市',  # 集美/海沧/同安/翔安区
    
    # 莆田市（35开头）
    '3511': '莆田市',  # 城厢/涵江区
    '3512': '莆田市',  # 荔城区/秀屿区
    '3513': '莆田市',  # 仙游县
    
    # 三明市（36开头）
    '3650': '三明市',  # 梅列/三元区
    '3651': '三明市',  # 永安市
    '3652': '三明市',  # 明溪县
    '3653': '三明市',  # 清流县
    '3654': '三明市',  # 宁化县
    '3655': '三明市',  # 大田县
    '3656': '三明市',  # 尤溪县
    '3657': '三明市',  # 沙县
    '3658': '三明市',  # 将乐县
    '3659': '三明市',  # 泰宁县
    '3660': '三明市',  # 建宁县
    
    # 泉州市（36开头）
    '3620': '泉州市',  # 鲤城/丰泽区
    '3621': '泉州市',  # 洛江区/泉港区
    '3622': '泉州市',  # 晋江市
    '3623': '泉州市',  # 南安市
    '3624': '泉州市',  # 石狮市
    '3625': '泉州市',  # 惠安县
    '3626': '泉州市',  # 安溪县
    '3627': '泉州市',  # 永春县
    '3628': '泉州市',  # 德化县
    
    # 漳州市（36开头）
    '3630': '漳州市',  # 芗城区
    '3631': '漳州市',  # 龙海市
    '3632': '漳州市',  # 云霄县
    '3633': '漳州市',  # 漳浦县
    '3634': '漳州市',  # 诏安县
    '3635': '漳州市',  # 长泰县
    '3636': '漳州市',  # 东山县
    '3637': '漳州市',  # 南靖县
    '3638': '漳州市',  # 平和县
    '3639': '漳州市',  # 华安县
    
    # 南平市（35开头）
    '3530': '南平市',  # 延平区
    '3531': '南平市',  # 邵武市
    '3532': '南平市',  # 武夷山市
    '3533': '南平市',  # 建瓯市
    '3534': '南平市',  # 建阳区
    '3535': '南平市',  # 顺昌县
    '3536': '南平市',  # 浦城县
    '3537': '南平市',  # 光泽县
    '3538': '南平市',  # 松溪县
    '3539': '南平市',  # 政和县
    
    # 龙岩市（36开头）
    '3640': '龙岩市',  # 新罗区
    '3641': '龙岩市',  # 漳平市
    '3642': '龙岩市',  # 长汀县
    '3643': '龙岩市',  # 永定区
    '3644': '龙岩市',  # 上杭县
    '3645': '龙岩市',  # 武平县
    '3646': '龙岩市',  # 连城县
    
    # 宁德市（35开头）
    '3521': '宁德市',  # 蕉城区
    '3522': '宁德市',  # 福鼎市
    '3523': '宁德市',  # 福安市
    '3524': '宁德市',  # 霞浦县
    '3525': '宁德市',  # 古田县
    '3526': '宁德市',  # 屏南县
    '3527': '宁德市',  # 寿宁县
    '3528': '宁德市',  # 周宁县
    '3529': '宁德市',  # 柘荣县
    
    # 特殊经济区（需归属到所在市）
    '350181': '福州市',  # 福清融侨开发区
    '362200': '泉州市',  # 晋江经济开发区
    '363109': '漳州市',   # 漳州招商局开发区

    # 南平市新增
    '3542': '南平市',  # 武夷山市
    '3544': '南平市',  # 建瓯市
    '3541': '南平市',  # 顺昌县
    '3543': '南平市',  # 浦城县
    
    # 宁德市新增
    '3552': '宁德市',  # 福鼎市
    '3551': '宁德市',  # 霞浦县
    '3554': '宁德市',  # 寿宁县
    '3555': '宁德市',  # 周宁县
    
    # 三明市新增
    '3661': '三明市',  # 永安市
    '3662': '三明市',  # 明溪县
    '3663': '三明市',   # 宁化县
    '3520': '南平市', '3545': '南平市', '3553': '宁德市', '3540': '南平市',
    '3550': '宁德市', '3556': '宁德市', '3543': '南平市', '3510': '莆田市',
    '3600': '厦门市'  # 注：360开头的厦门旧编码已逐步被361替代

    }
    return pd.DataFrame({
        'postcode_prefix': data.keys(),
        'city': data.values()
    })

# 2. 创建市级查询器
class CityMatcher:
    def __init__(self):
        self.df = create_fujian_city_map()
        self.mapping = self.df.set_index('postcode_prefix')['city'].to_dict()
    
    def get_city(self, postcode):
        """根据邮编返回地级市名称"""
        if pd.isna(postcode):
            return "未知地区"
        
        try:
            prefix = str(int(float(postcode)))[:4]  # 处理各种格式的邮编
            return self.mapping.get(prefix, "未知地区")
        except:
            return "未知地区"

# 3. 使用示例
if __name__ == "__main__":
    # 初始化匹配器
    matcher = CityMatcher()
    # 应用查询
    df_z["市"] = df_z["申请人邮编"].apply(matcher.get_city)
    
    print("测试结果:")
    print(df_z)
    

测试结果:
                    申请号       公开（公告）号         申请日     公开（公告）日  \
0      CN202410012063.2  CN118135171A  2024.01.03  2024.06.04   
1      CN202410015335.4  CN118038893A  2024.01.05  2024.05.14   
2      CN202410048496.3  CN117833943A  2024.01.12  2024.04.05   
3      CN202410032903.1  CN117855767A  2024.01.09  2024.04.09   
4      CN202410038215.6  CN117896881A  2024.01.10  2024.04.16   
...                 ...           ...         ...         ...   
27333  CN202411989720.5  CN119889437A  2024.12.31  2025.04.25   
27334  CN202411996494.3  CN119812685A  2024.12.31  2025.04.11   
27335  CN202411997100.6  CN119541383A  2024.12.31  2025.02.28   
27336  CN202411997340.6  CN119694253A  2024.12.31  2025.03.25   
27337  CN202510227129.4  CN119862605A  2024.12.16  2025.04.22   

                                                  IPC分类号  \
0      G06N3/08;G06N3/0464;G06V20/70;G06V10/10;G06V10/82   
1                G10L25/03;G10L15/06;G10L25/27;G10L25/51   
2                                

In [ ]:
df_z = df_z[df_z['市']!='未知地区']
df_z

,申请号,公开（公告）号,申请日,公开（公告）日,IPC分类号,申请（专利权）人,发明名称,申请人邮编,文献类型,市
0,CN202410012063.2,CN118135171A,2024.01.03,2024.06.04,G06N3/08;G06N3/0464;G06V20/70;G06V10/10;G06V10/82,新大陆数字技术股份有限公司,一种基于深度学习的磁翻板液位仪自动预警方法,350015,发明,福州市
1,CN202410015335.4,CN118038893A,2024.01.05,2024.05.14,G10L25/03;G10L15/06;G10L25/27;G10L25/51,厦门快商通科技股份有限公司,基于声纹聚类的阈值选取方法、声纹聚类方法及装置,361000,发明,厦门市
2,CN202410048496.3,CN117833943A,2024.01.12,2024.04.05,H04B1/16,数字蓝盾（厦门）信息科技有限公司,一种基于软件定义无线电的宽带信号接收装置及接收处理方法,361101,发明,厦门市
3,CN202410032903.1,CN117855767A,2024.01.09,2024.04.09,H01M50/586;H01M50/178;B32B27/08;B32B7/12;H01M5...,宁德新能源科技有限公司,绝缘件、极耳组件、电芯、电池及用电设备,352100,发明,宁德市
4,CN202410038215.6,CN117896881A,2024.01.10,2024.04.16,H05B45/30;H05B45/50;H05B47/155;H02J7/00,泉州海川半导体有限公司,一种用于移动电源外部LED灯数量的自动识别方法,362100,发明,泉州市
...,...,...,...,...,...,...,...,...,...,...
27333,CN202411989720.5,CN119889437A,2024.12.31,2025.04.25,G16B20/00;G16B20/20;G16B20/50,厦门艾德生物医药科技股份有限公司,用于NGS样本污染的检测方法及其系统,361027,发明,厦门市
27334,CN202411996494.3,CN119812685A,2024.12.31,2025.04.11,H05K1/02;H05K1/14;H01M50/503;H01M50/519;H01R12/71,厦门新能达科技有限公司,电池组及用电装置,361000,发明,厦门市
27335,CN202411997100.6,CN119541383A,2024.12.31,2025.02.28,H10H29/49;G09G3/32;H10D86/60;H10H29/30;H10K59/...,天马新型显示技术研究院（厦门）有限公司; 厦门天马微电子有限公司,一种阵列基板、显示面板及拼接显示屏,361101,发明,厦门市
27336,CN202411997340.6,CN119694253A,2024.12.31,2025.03.25,G09G3/3225,厦门天马显示科技有限公司,显示面板及显示装置,361101,发明,厦门市


In [28]:
df_z.to_csv("D:\文档\坚果云\区域经济\数字经济\数字经济指数\数据\专利\福建数字专利2024_分城市.csv", index=False)

In [25]:
df_z[df_z['市']=='未知地区']['申请人邮编'].unique()

array([511300, 510000, 221600,  56599, 223001, 528000, 518000, 221000,
       510442, 210000, 751100, 250023, 255000, 214000, 311100, 277300,
       211700, 665000, 213000, 256600, 226600, 518052, 360100, 300380])

In [23]:
df_2022 = pd.read_stata(r"E:\Data\数字专利\2022.dta",chunksize=100)
df_2022.__next__()

,newipzlid,年份,申请日,标题,摘要,申请人,公开公告号,公开公告日,申请号,专利类型,...,优先权号,优先权日,授权公告号,授权公告日,省,省代码,市,市代码,县,县代码
0,20220000001,2022,2022-01-11,一种直播交互方法、装置、电子设备及计算机程序产品,本申请实施例提供了一种直播交互方法、装置及电子设备。其中，直播交互方法包括：提供一直播界面，...,杭州阿里云飞天信息技术有限公司,CN114051172B,2024-03-22,CN202210024435.4,发明授权,...,,,CN114051172B,2024-03-22,浙江省,330000.0,杭州市,330100.0,余杭区,330110.0
1,20220000002,2022,2022-01-05,主站系统的定制值流转方法,本发明公开了主站系统的定制值流转方法，包括步骤：站端人员操作终端设备，接收任务，进行定值的校...,国网江苏省电力有限公司淮安供电分公司; 国网江苏省电力有限公司,CN114158217B,2024-03-22,CN202210006946.3,发明授权,...,CN2021109066752,2021-08-09,CN114158217B,2024-03-22,江苏省,320000.0,淮安市,320800.0,清江浦区,320812.0
2,20220000005,2022,2022-01-07,一种低温相变蓄冷材料,本发明公开了一种低温相变蓄冷材料，涉及一种有机‑无机复合相变材料，包括无水氯化钙、蒸馏水、丙...,北部湾大学,CN114181672B,2024-03-22,CN202210016878.9,发明授权,...,,,CN114181672B,2024-03-22,广西壮族自治区,450000.0,钦州市,450700.0,钦南区,450702.0
3,20220000007,2022,2022-01-04,一种功能性墙布及其制备方法,本发明公开了一种功能性墙布及其制备方法，包括制备聚丙烯酰胺改性HDPE树脂、制备改性HDPE...,浙江红饰和家居有限公司,CN114193869B,2024-03-22,CN202210000996.0,发明授权,...,,,CN114193869B,2024-03-22,浙江省,330000.0,绍兴市,330600.0,越城区,330602.0
4,20220000008,2022,2022-01-14,包含预测与滤波的间接手术机器人远程控制方法,本发明涉及一种包含预测与滤波的间接手术机器人远程控制方法，其解决微创手术机器人远程手术过程中...,山东威高手术机器人有限公司,CN114209435B,2024-03-22,CN202210040019.3,发明授权,...,,,CN114209435B,2024-03-22,山东省,370000.0,威海市,371000.0,环翠区,371002.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20220000187,2022,2022-01-17,一种育肥羊生物饲料及其制备方法和应用,本发明提供一种育肥羊生物饲料及其制备方法和应用，涉及动物饲料领域。本发明提供的酿酒酵母菌具体...,沈阳丰美生物技术有限公司,CN117730951A,2024-03-22,CN202311452591.1,发明申请,...,,,,,辽宁省,210000.0,沈阳市,210100.0,浑南区,210112.0
96,20220000188,2022,2022-01-24,虚拟场景的回放方法、介质、电子设备及计算机程序产品,一种虚拟场景的回放方法、介质、电子设备及计算机程序产品。该方法用于电子设备，该方法包括：接收...,上海莉莉丝科技股份有限公司,CN117750999A,2024-03-22,CN202280027358.6,发明申请,...,,,,,上海市,310000.0,上海市,310000.0,嘉定区,310114.0
97,20220000191,2022,2022-01-25,一种整流器控制的方法以及装置,本申请实施例提供一种整流器控制的方法和装置。该方法包括：根据三相交流信号的采样电压和采样电流...,宁德时代新能源科技股份有限公司,CN117751516A,2024-03-22,CN202280054822.0,发明申请,...,,,,,福建省,350000.0,宁德市,350900.0,蕉城区,350902.0
98,20220000192,2022,2022-01-26,一种钙钛矿太阳能电池、其封装方法及包含其的光伏组件,本申请提供一种钙钛矿太阳能电池、其封装方法以及包含其的光伏组件。所述钙钛矿太阳能电池包括从下...,宁德时代新能源科技股份有限公司,CN117751699A,2024-03-22,CN202280052612.8,发明申请,...,,,,,福建省,350000.0,宁德市,350900.0,蕉城区,350902.0


In [4]:
df_2022.__next__().columns

Index(['newipzlid', '年份', '申请日', '标题', '摘要', '申请人', '公开公告号', '公开公告日', '申请号',
       '专利类型', '公开国别', '首项权利要求', '独立权利要求', '文献页数', 'IPC主分类', 'IPC', '洛迦诺分类号',
       '当前权利人', '申请人类型', '申请人国家_地区', '申请人地址', '当前专利权人地址', '工商注册地址', '工商公司类型',
       '工商成立日期', '工商统一社会信用代码', '工商注册号', '工商上市代码', '工商企业状态', '发明人', '引证次数',
       '被引证次数', '自引次数', '他引次数', '被自引次数', '被他引次数', '家族引证次数', '家族被引证次数', '优先权信息',
       '优先权号', '优先权日', '授权公告号', '授权公告日', '省', '省代码', '市', '市代码', '县', '县代码'],
      dtype='object')

In [8]:
import os
output_file = 'E:\Data\数字专利\福建数字专利2023.csv'
columns_to_save = ['申请日', '标题','申请人', '公开公告号', '公开公告日', '申请号','专利类型','市','省']
if not os.path.exists(output_file):
    pd.DataFrame(columns=columns_to_save).to_csv(output_file, index=False, header=True)
for chunk in pd.read_stata(r"E:\Data\数字专利\2023.dta",chunksize=1000):
    chunk[chunk['省']=='福建省'][columns_to_save].to_csv(output_file, mode='a', index=False, header=False)